# Train Deep-HiTS

### Deep-HiTS model from paper, trainning under LRP framework to save model's parameters 

#### Imports

In [2]:
#python 2 and 3 comptibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#enables acces to parent folder ()
import os
import sys
sys.path.append("../../..")

#other imports
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

#modules import
import modules.sequential32 as sequential32
import modules.linear32 as linear32
import modules.convolution32 as convolution32
import modules.maxpool32 as maxpool32
import modules.rotation as rotation


#print of plots within notebook
%matplotlib inline

#### Trainning Parameters

In [3]:
#number of samples for every trainning iteration
BATCH_SIZE = 50
#First 100k iterations of trainning
INITIAL_PATIENCE = 100000  
#Validate trainning every 10k iterations
VALIDATION_PERIOD = 10000  
#Update learning rate every 100k iterations
ANNEALING_PERIOD = 100000  
#If validation criteria is met, increase trainning iterations by 100k
PATIENCE_INCREMENT = 100000
#path where summaries, graph and best  trained models will be saved
SUMMARY_DIR = "TB/deep_hits_runs/"
#path to *.tfrecord files (data) in your computer
whole_data_path = '/home/tesla/Desktop/LRP/deep_hits_data/'

#### Functions to acces data base

To do this, you must have HiTS 2013 database in tfrecords format; train (1.2 million samples), validation (100k samples) and test (100k samples) set. You can finde them on __TODO__....

In [4]:
#funtion to get certain dataset tensor
def get_dataset_tensor(dataset):
    data_path = whole_data_path+dataset
    feature = {'image_raw': tf.FixedLenFeature([], tf.string),
               'label': tf.FixedLenFeature([], tf.int64),
               'snr': tf.FixedLenFeature([], tf.string)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path])
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image_raw'], tf.float32)

    # Cast label data into int32
    label = tf.cast(features['label'], tf.int32)
    
    # Cast snr data into int32
    snr = tf.decode_raw(features['snr'], tf.float64)
    
    # Reshape image data into the original shape
    image = tf.reshape(image, [21, 21, 4])
    
    snr = tf.reshape(snr, [1])

    # Any preprocessing here ...

    # Creates batches by randomly shuffling tensors
    images, labels, snrs= tf.train.batch([image, label, snr],
                                    batch_size=BATCH_SIZE,
                                    capacity=100000,
                                    num_threads=1)
    return images, labels, snrs

def get_train_tensors():
    return get_dataset_tensor('snr_train.tfrecord')


def get_validation_tensors():
    return get_dataset_tensor('snr_validation.tfrecord')


def get_test_tensors():
    return get_dataset_tensor('snr_test.tfrecord')

#### Function to perform rotation
Define function for rotate tensor in [0,90,180,270] degrees

In [5]:
"""
given an image batch tensor as [batch_size,height,width,channels], 
generate 4 rotated versions in [0,90,180,270] degrees and concatenate them in batch dimension 

@param img_batch input tensor thath contains images
@return a [4*batch_size,height,width,channels] version of img_batch
"""
def augment_with_rotations(img_batch):   
    #perform rotations
    images90 = tf.map_fn(lambda x: tf.image.rot90(x, k=1), img_batch)
    images180 = tf.map_fn(lambda x: tf.image.rot90(x, k=2), img_batch)
    images270 = tf.map_fn(lambda x: tf.image.rot90(x, k=3), img_batch)
    
    #concatenate along first tensor dimension (batch dimension)
    return tf.concat([img_batch,
                         images90,
                         images180,
                         images270], 0)

#### Define network model

Deep-HiTS model for LRP framework, exact same as in paper. It considers:

1. Leaky-ReLU
2. He weights initialization
3. Biases initializated in 0.0
4. Stretching layer after convolutions that concatenates features

In [6]:
def nn():
    return sequential32.Sequential([
        
        convolution32.Convolution(kernel_size=4, output_depth=32,            
                                  input_depth=4, input_dim=27, act ='lrelu',    
                                  stride_size=1, pad='VALID',                         
                                  weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(27*27*4))),
                                  bias_init= tf.constant_initializer(0.0)),
                                
                                    
        convolution32.Convolution(kernel_size=3, output_depth=32, stride_size=1, act ='lrelu', pad='SAME',
                                  weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(24*24*32))),
                                  bias_init= tf.constant_initializer(0.0)),

                       
        maxpool32.MaxPool(),

        convolution32.Convolution(kernel_size=3, output_depth=64, stride_size=1, act ='lrelu', pad='SAME',
                                  weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(12*12*32))),
                                  bias_init= tf.constant_initializer(0.0)),

                                    
        convolution32.Convolution(kernel_size=3, output_depth=64, stride_size=1, act ='lrelu', pad='SAME',
                                  weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(12*12*64))),
                                  bias_init= tf.constant_initializer(0.0)),

                                    
        convolution32.Convolution(kernel_size=3, output_depth=64, stride_size=1, act ='lrelu', pad='SAME',
                                  weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(12*12*64))),
                                  bias_init= tf.constant_initializer(0.0)),

        maxpool32.MaxPool(),
                                    
        rotation.Rotation(rotation_num=4),
                       
        linear32.Linear(64, act ='lrelu', keep_prob=keep_prob, use_dropout = True,
                        weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(9216))),
                        bias_init= tf.constant_initializer(0.0)),
                                               
        linear32.Linear(64, act ='lrelu', keep_prob=keep_prob, use_dropout = True,
                        weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(64))),
                        bias_init= tf.constant_initializer(0.0)),

        linear32.Linear(2, act ='linear',
                        weights_init= tf.truncated_normal_initializer(stddev=np.sqrt(2/(64))),
                        bias_init= tf.constant_initializer(0.0))])



#### Model I/O

In [7]:
#Dropout placeholder
keep_prob = tf.placeholder(tf.float32, name='keep-prob')

#Input place holders and variables
with tf.device('/cpu:0'):
    #train tensors variables, run them to get dataset sample
    images, labels, snr = get_train_tensors()
    #validation tensors variables
    validation_images, validation_labels, validation_snr = get_validation_tensors()
    #test tensors variables
    test_images, test_labels, test_snr = get_test_tensors()
    
    #place holder with default for input image, it enables to pass an specific tensor,
    #or call 'images' to get a train tensor
    images = tf.placeholder_with_default(images,
                                         (None, 21, 21, 4),
                                         'images_placeholder')
    #place holder with default for input label, it enables to pass an specific label,
    #or call 'labels' to get a train label
    labels = tf.placeholder_with_default(labels,
                                         None,
                                         'labels_placeholder')
    #label to one-hot
    one_hot_labels = tf.one_hot(labels, 2, dtype=tf.float32)
    #generate rotated images
    augmented_input = augment_with_rotations(images)
    #zero-pad images from 21x21 stamps to 27x27, to fit a 4x4 filter
    padded_input = tf.pad(augmented_input,
                          paddings=[
                              [0, 0],
                              [3, 3],
                              [3, 3],
                              [0, 0]])
    
    #input to model
    inp=padded_input
    #labels
    y_=one_hot_labels

#Model instance
with tf.variable_scope('model'):
        #instanciate model
        net = nn()       
        #feed-forward method and get score output. This iterates through layer in net object of class Sequiential
        score = net.forward(inp)
        #pass scores through softmax for network output ([0,1] probability)   
        y = tf.nn.softmax(score)
        #predicted classes
        y_pred_cls = tf.argmax(y, 1)
        #true classes
        y_true_cls = tf.argmax(y_, 1)

Forward Pass ... 
------------------------------------------------- 
input:: [None, 27, 27, 4]
conv2d_1:: [None, 24, 24, 32]
conv2d_2:: [None, 24, 24, 32]
maxpool_3:: [None, 12, 12, 32]
conv2d_4:: [None, 12, 12, 64]
conv2d_5:: [None, 12, 12, 64]
conv2d_6:: [None, 12, 12, 64]
maxpool_7:: [None, 6, 6, 64]
rotation_8:: [None, 9216]
linear_9:: [None, 64]
linear_10:: [None, 64]
linear_11:: [None, 2]
softmax:: [None, 2]

------------------------------------------------- 


#### Trainning Variables

In [8]:
#trainning variables
#Enable learning rate actualization through trainning
with tf.variable_scope('train'):
        #loss function
        diff = tf.nn.softmax_cross_entropy_with_logits(logits=score, labels=y_)
        #cost function
        cost = tf.reduce_mean(diff)
        #learning rate that will be actuallized every ANNEALING_PERIOD iterations
        learning_rate = tf.Variable(0.04, trainable=False, collections=[])
        #SDG optimizer
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        
        #train operation to be run for performing a leraning iteration
        train_op = optimizer.minimize(cost)

#### Accuracy measurement 
Build accuracy and cost list for summary

In [9]:
with tf.name_scope('accuracy'):  
        #compare predictions
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(y_pred_cls, y_true_cls)
        #get accuracy
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
#tensorflow summary        
accuracy_sum = tf.summary.scalar('accuracy', accuracy)

#metrics to be used on validation criteria
metrics = (cost, accuracy)
#names for those metrics
metrics_names = ('xentropy', 'accuracy')
#list of metrics as a TF summary
metric_summary_list = []

#add metrics to summary list
for metric, name in zip(metrics, metrics_names):
    summary = tf.summary.scalar(name, metric)
    metric_summary_list.append(summary)

#get unique summary for validation criteria metrics
merged_summaries = tf.summary.merge(metric_summary_list)

#### Other performance measures
Explicit metrics to build Table II of paper

In [10]:
with tf.name_scope('performance_measures'):
    #confusion matrix values
    with tf.name_scope('values'):
        #true positives
        TP = tf.count_nonzero((y_pred_cls * y_true_cls))
        #true negative
        TN = tf.count_nonzero((y_pred_cls - 1) * (y_true_cls- 1))
        #false positives
        FP = tf.count_nonzero(y_pred_cls * (y_true_cls - 1))
        #false negatives
        FN = tf.count_nonzero((y_pred_cls - 1) * y_true_cls)
        
    with tf.name_scope('accuracy_func'):
            acc_mes = (TP+TN)/(TP+TN+FN+FP)
        
    with tf.name_scope('precision_func'):
            prec_mes = TP/(TP+FP)
            
    with tf.name_scope('recall_func'):
            rec_mes = TP/(TP+FN)      
            
    with tf.name_scope('f1_func'):
            f1_mes = 2 * prec_mes * rec_mes / (prec_mes + rec_mes) 

#### Init Tensorflow session

In [11]:
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

#### Init TensorBoard writers, Model saver, Model variables and Tfrecords coordinators

In [12]:
train_writer = tf.summary.FileWriter(os.path.join(SUMMARY_DIR, 'train'),
                                     sess.graph,
                                     max_queue=1000)
validation_writer = tf.summary.FileWriter(os.path.join(SUMMARY_DIR, 'validation'))

saver = tf.train.Saver()

#initialization of model's variables
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Create a coordinator and run all QueueRunner objects for parallel loading of tfrecords
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

## Definning train, validation and test functions

#### Validation Function

In [13]:
"""
Called by trainning functiontion to validate model, it gets accuracy and loss
of model in validation set (100k samples), save best model so far and verify validation criteria. 
If criteria is met, another PATIENCE_INCREMENT iterations of trainning will be performed.

@param current_iteration
@param current_patience trainning iterations left 
@param best_model accuracy and loss of best model so far
@param stopping_criteria_model loss and accuracy of last model that met validation criteria
@return current_patience or new_patience (current_iterations+PATIENCE_INCREMENT) if validation criteria
is met 
"""
def validate(current_iteration, current_patience, best_model, stopping_criteria_model):
    #check if current_iteration is multiple of VALIDATION_PERIOD
    #to perform validation every VALIDATION_PERIOD iterations
    if current_iteration % VALIDATION_PERIOD != 0:
        return current_patience
    
    #to store validation accuracy and loss
    metric_data = {}
    for metric in metrics:
        metric_data[metric] = {
            'values_per_batch': [],
            'batch_mean': None
        }

    #Validate validation set.
    for val_batch in range(100000 // BATCH_SIZE):
        #get validation batch from tfrecords
        images_array, labels_array = sess.run((
            validation_images,
            validation_labels))
        #get accuracy and loss
        metrics_value = sess.run(metrics,
                                 feed_dict={
                                     keep_prob: 1.0,
                                     images: images_array,
                                     labels: labels_array
                                 })
        #append every batch metric to metric_data
        for metric, value in zip(metrics, metrics_value):
            metric_data[metric]['values_per_batch'].append(value)
            
    #get accuracy and loss for al validation batches and write them to validation writer
    #as a summary
    for metric, name in zip(metrics, metrics_names):
        metric_data[metric]['batch_mean'] = np.array(metric_data[metric]['values_per_batch']).mean()
        summary = tf.Summary()
        summary.value.add(tag=name, simple_value=float(metric_data[metric]['batch_mean']))
        validation_writer.add_summary(summary, current_iteration)
    
    #mean accuracy of validation set
    accuracy_mean = metric_data[accuracy]['batch_mean']
    #Check if accuracy is over best model so far and overwrite model checkpoint   
    if accuracy_mean > best_model['accuracy']:
        best_model['accuracy'] = accuracy_mean
        best_model['iteration'] = current_iteration
        print("New best model: Accuracy %.4f @ it %d" % (
            best_model['accuracy'],
            best_model['iteration']
        ))
        #ckpt_dir = os.path.join(SUMMARY_DIR, 'ckpt_files')
        #if not os.path.exists(ckpt_dir):
        #    os.makedirs(ckpt_dir)
        saver.save(sess, SUMMARY_DIR)

    #check stopping criteria, which is:
    #if current model error is under 99% of the error of 
    #last model that met stopping criteria, validation criteria is met
    if (1.0-accuracy_mean) < 0.99*(1.0-stopping_criteria_model['accuracy']):
        stopping_criteria_model['accuracy'] = accuracy_mean
        stopping_criteria_model['iteration'] = current_iteration
        new_patience = current_iteration + PATIENCE_INCREMENT
        if new_patience > current_patience:
            print("Patience increased to %d because of model with accuracy %.4f @ it %d" % (
                new_patience,
                stopping_criteria_model['accuracy'],
                stopping_criteria_model['iteration']
            ))
            return new_patience
        else:
            return current_patience
    else:
        return current_patience

#### Test Function

In [14]:
"""
Evaluate model on specific set.

@param set_images
@param set_labels
@return loss, accuracy, precision, reacall and f1_score of model
"""
def eval_set(set_images, set_labels):
    #create list of metric to store per batch evaluation
    losses = []
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    
    #get metrics on every batch
    for test_batch in range(100000 // BATCH_SIZE):
        images_array, labels_array = sess.run((
            set_images,
            set_labels))
        loss_val, acc_val, prec, rec, f1 = sess.run((cost, acc_mes, prec_mes, rec_mes, f1_mes),
                                     feed_dict={
                                         keep_prob: 1.0,
                                         images: images_array,
                                         labels: labels_array
                                     })
        losses.append(loss_val)
        accuracies.append(acc_val)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        
    #average all batch metric on a single one
    loss_mean = np.array(losses).mean()
    accuracy_mean = np.array(accuracies).mean()
    precision_mean = np.array(precisions).mean()
    recall_mean = np.array(recalls).mean()
    f1_mean = np.array(f1s).mean()
    

    return loss_mean, accuracy_mean, precision_mean, recall_mean, f1_mean

"""
Evaluate model on test set (100k samples).

@return loss, accuracy, precision, reacall and f1_score of model ofer test
"""
def test():
    return eval_set(test_images, test_labels)

#### Learning rate update and trainning functions

In [15]:
def update_learning_rate(global_step):
    sess.run(tf.assign(learning_rate,
                       0.04/(2.0**(global_step//ANNEALING_PERIOD))))
    lr_value = sess.run(learning_rate)
    print("Iteration %d. Learning rate: %.4f" % (global_step, lr_value))

"""
Train model from scratch, it means to reinitialice model parameters (weights and biases), 
set current iteration to 0 (global_step), and initialize best_model and stopping_criteria_model
to 0.5 accuracy. After trainning process restores best model and it perform an evaluation over test set.


@return test_accuracy, test_prec, test_rec, test_f1 of best trainning model
"""
def train_from_scratch():
    #initialize params
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    #init current it to 0
    global_step = 0
    patience = INITIAL_PATIENCE
    #init models acc to 0.5
    best_model = {
        'iteration': 0,
        'accuracy': 0.5
    }
    stopping_criteria_model = {
        'iteration': 0,
        'accuracy': 0.5
    }
    #train model
    while global_step < patience:
        #check if learning rate must be updated
        if global_step % ANNEALING_PERIOD == 0:
            update_learning_rate(global_step)
        #perform a trainning iteration
        summaries_output, _ = sess.run((merged_summaries,
                                        train_op),
                                       feed_dict={
                                           keep_prob: 0.5
                                       })
        global_step += 1
        #trainning writer is commented due large amount of data that it stores
        #write accuracy and loss summaries to train writer
        #train_writer.add_summary(summaries_output, global_step)
        
        #validate model after trainning iteration
        patience = validate(global_step, patience, best_model, stopping_criteria_model)
    
    #restore best model so far
    saver.restore(sess, SUMMARY_DIR)
    #evaluate model over test set
    test_loss, test_accuracy, test_prec, test_rec, test_f1 = test()
    print("Best model @ it %d.\nValidation accuracy %.5f, Test accuracy %.5f" % (
        best_model['iteration'],
        best_model['accuracy'],
        test_accuracy
    ))
    print("Test loss %.5f" % test_loss)
    
    return test_accuracy, test_prec, test_rec, test_f1

### Train model

Here you can modify variable N_models to train as many models as you want. The trainning will be sequential, and at the beginning of a new trainning, parameters of last model will be erased and reinitializated. Test and Validation metrics of every trained model are stored in lists.

In [16]:
#Numbers of model to train
N_models = 1

#list of test set parameters to save of each model
acc_ls = []
prec_ls = []
rec_ls = []
f1_ls = []

#list of validation set parameters to save of each model
acc_ls_val = []
prec_ls_val = []
rec_ls_val = []
f1_ls_val = []


for i in range(N_models):
        
    print("\nModel:", i)
    test_accuracy, test_prec, test_rec, test_f1 = train_from_scratch()
        
    acc_ls.append(test_accuracy)
    prec_ls.append(test_prec)
    rec_ls.append(test_rec)
    f1_ls.append(test_f1)
    
    _, val_accuracy, val_prec, val_rec, val_f1 = eval_set(validation_images, validation_labels)
    
    acc_ls_val.append(val_accuracy)
    prec_ls_val.append(val_prec)
    rec_ls_val.append(val_rec)
    f1_ls_val.append(val_f1)


Model: 0
Iteration 0. Learning rate: 0.0400


KeyboardInterrupt: 

### Print Validation and Test Metrics means and stds of all models

In [ ]:
"""
get mean and std of a list of models parameters

@metric_ls list of metrics
@metric_name
@return metric_mean
@return metric_std
"""
def getMetricVal(metric_ls, metric_name):
    metric_mean = np.array(metric_ls).mean()*100
    metric_std = np.array(metric_ls).std()*100
    print("%s %.2f +/- %.2f" % (
        metric_name,
        np.array(metric_ls).mean()*100,
        np.array(metric_ls).std()*100
    ))
    return metric_mean, metric_std

print("Test Metrics\n")
_,_=getMetricVal(acc_ls, 'Accuracy')

_,_=getMetricVal(prec_ls, 'Precision')

_,_=getMetricVal(rec_ls, 'Recall')

_,_=getMetricVal(f1_ls, 'F1 Score')
    

In [ ]:
print("Val Metrics\n")
_,_=getMetricVal(acc_ls_val, 'Accuracy')

_,_=getMetricVal(prec_ls_val, 'Precision')

_,_=getMetricVal(rec_ls_val, 'Recall')

_,_=getMetricVal(f1_ls_val, 'F1 Score')

### Get and save Models weights and biases

This will save latest trained model params, each of them will be saved as a '\*.npy' file

In [ ]:
#get parameters
W,B = net.getWeights()
weights, biases = sess.run([W,B])

In [ ]:
#save parameters on params_path
params_path = '/home/esteban/Esteban/LRPbeta/Code_vFloat32_TB/weights/deephits/'

"""
Save weights to *.npy files, where format is 'typeOfLayer_layerNumber_typeOfParameter.npy', for example
for the first fully-connected layer weights will have 'FC1-W.npy'.

@Ncnn number of convolutional layers with params
@Nfc number of dense layers with params
@weights weights from model
@biases biases from model
"""
def saveWeights(Ncnn, Nfc, weights, biases):
    for i in range(Ncnn):
        np.save(params_path+'CNN'+str(i+1)+'-W.npy', np.array(weights[i]))
        np.save(params_path+'CNN'+str(i+1)+'-B.npy', np.array(biases[i]))

    for i in range(Nfc):
        np.save(params_path+'FC'+str(i+1)+'-W.npy', np.array(weights[i+Ncnn]))
        np.save(params_path+'FC'+str(i+1)+'-B.npy', np.array(biases[i+Ncnn]))

#save model params
saveWeights(5,3,weights,biases)

## Get softmax outputs to build DET curve

This consumes alot of memory, we don't know why yet, but we belive that there is a memory leak in the framework.

Don't run this cell if not necessary

In [ ]:
def getParams4Metrics( set_images, set_labels, set_snr):
    
        lbls = []
        probs = []
        snr = []
        for test_batch in range(100000 // BATCH_SIZE):
            images_array, labels_array, snr_array = sess.run((
                set_images,
                set_labels,
                set_snr))
            probs_val = sess.run((y[:,1]), feed_dict={
                                             keep_prob: 1.0,
                                             images: images_array,
                                             labels: labels_array,
                                             phase_train: False
                                         })
            lbls.append(labels_array)
            probs.append(probs_val)
            snr.append(snr_array)
            print(test_batch)

        return np.concatenate(lbls).astype(int), np.concatenate(probs), np.concatenate(snr)
    
lbls, probs, snrs = getParams4Metrics(validation_images, validation_labels, validation_snr) 


np.save('/home/ayudante/Desktop/Esteban/DETparams/DHExact/val.npy', np.stack((lbls, probs, snrs[:,0])))

In [ ]:
lbls, probs, snrs = getParams4Metrics(test_images, test_labels, test_snr)

np.save('/home/ayudante/Desktop/Esteban/DETparams/DHExact/test.npy', np.stack((lbls, probs, snrs[:,0])))

# TODO

# - Liberate HiTS 2013 tfrecords
# __TODO__ cite paper
# score = tf.squeeze(score)  #TODO delete
# __TODO__ reference paper our
# TB